# Gestión de una Biblioteca

## Descripción del Proyecto

El objetivo principal del proyecto es crear una aplicación para gestionar una biblioteca. La aplicación permitirá realizar diversas operaciones, como la gestión de libros, autores y usuarios, así como la realización y seguimiento de préstamos de libros.

#### Conexión a la base de datos y creación del cursor
**Objetivo:** Establecer una conexión con una base de datos SQLite y crear las tablas necesarias para la aplicación.

In [70]:
def connect_to_db():
    import sqlite3
    try: 
        # Solicitar nombre de la base de datos
        database = input("Nombre de la base de datos (Intente con 'Biblioteca.db'): ")
        # Conectar a la base de datos
        conn = sqlite3.connect(database) 
        # Crear un cursor
        cursor = conn.cursor() 
        # Devolver la conexión y el cursor para su uso posterior
        print("Se realizó la conexión correctamente y se creó un cursor")
        return conn, cursor
    except sqlite3.DatabaseError as error:
        print(f"Error de base de datos de SQLite: {error}")
    except Exception as e:
        print(f"Ups, algo no salió bien. Intente todo otra vez. Detalle del error: {e}")

connect_to_db()

Se realizó la conexión correctamente y se creó un cursor


(<sqlite3.Connection at 0x1207023e0>, <sqlite3.Cursor at 0x120c15f40>)

#### Creación de las tablas

In [ ]:
# Libros
cursor.execute('''
CREATE TABLE  books (
    isbn INTEGER,
    title VARCHAR NOT NULL,
    authors VARCHAR,
    categories VARCHAR NOT NULL,
    description VARCHAR NULL,
    published_year INTEGER NOT NULL,
    num_pages INTEGER NOT NULL,
    status VARCHA NOT NULL,

CONSTRAINT isbn_pk PRIMARY KEY (isbn),
CONSTRAINT status_chk CHECK (status IN ('Available', 'Issued')),
CONSTRAINT fk_authors FOREIGN KEY (authors) REFERENCES authors(name)
)''')

In [ ]:
# Usuario
cursor.execute('''
CREATE TABLE users (
    user_id INTEGER PRIMARY KEY AUTOINCREMENT,
    username VARCHAR NOT NULL,
    email VARCHAR UNIQUE NOT NULL,
    phone VARCHAR,
    join_date DATE NOT NULL,
    status VARCHAR CHECK (status IN ('Active', 'Inactive'))
)''')

In [69]:
# Autores
cursor.execute('''
CREATE TABLE  authors (
    author_id VARCHAR,
    names VARCHAR,
    title VARCHAR NOT NULL,
    published_year INTEGER NOT NULL,

CONSTRAINT author_id_pk PRIMARY KEY (author_id)
)''')

In [ ]:
#Préstramos
cursor.execute('''
CREATE TABLE loans (
    loan_id INTEGER PRIMARY KEY AUTOINCREMENT,
    isbn INTEGER,
    author VARCHAR,               
    user_id VARCHAR,
    loan_date DATE NOT NULL,
    return_date DATE NOT NULL,

FOREIGN KEY (isbn) REFERENCES books(isbn),
FOREIGN KEY (author) REFERENCES authors(name),
FOREIGN KEY (user_id) REFERENCES users(user_id),
CHECK (return_date IS NULL OR return_date >= loan_date)
)''')

### Gestión de Atributos de la biblioteca

 **Objetivo:** Implementar funcionalidades para agregar, obtener y eliminar registros en la base de datos.

 Se añadió una base de datos de libros preexistente para demostración de funcionalidades


In [59]:
# Insertar el DataFrame en la base de datos
import pandas as pd
import numpy as np

books = pd.read_csv('/Users/bixo/Desktop/Programming/Python PROTECO/Intermedio/Proyecto_Intermedio/books.csv')

# Eliminar columnas no necesarias
books = books.drop(columns=['isbn10', 'thumbnail', 'average_rating', 'ratings_count', 'subtitle'])

#Adición de una columna de estado (Prestado/Disponible)
n = len(books) #Tamaño de 'books'
probabilities = [0.9, 0.1]  # 90% Available, 10% Issued

# Crear una columna con valores aleatorios True o False
books['status'] = np.random.choice(['Available', 'Issued'], size=n, p=probabilities)
display(books)


database = input("Nombre de la base de datos (Intente con 'Biblioteca.db'): ")
        # Conectar a la base de datos
conn = sqlite3.connect(database) 
        # Crear un cursor
cursor = conn.cursor() 


#Añadir 
books.to_sql('books', conn, if_exists='replace', index=False)

,isbn13,title,authors,categories,description,published_year,num_pages,status
0,9780002005883,Gilead,Marilynne Robinson,Fiction,A NOVEL THAT READERS and critics have been eag...,2004.0,247.0,Available
1,9780002261982,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,A new 'Christie for Christmas' -- a full-lengt...,2000.0,241.0,Available
2,9780006163831,The One Tree,Stephen R. Donaldson,American fiction,Volume Two of Stephen Donaldson's acclaimed se...,1982.0,479.0,Available
3,9780006178736,Rage of angels,Sidney Sheldon,Fiction,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,512.0,Available
4,9780006280897,The Four Loves,Clive Staples Lewis,Christian life,Lewis' work on the nature of love divides love...,2002.0,170.0,Available
...,...,...,...,...,...,...,...,...
6805,9788185300535,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,This collection of the timeless teachings of o...,1999.0,531.0,Available
6806,9788185944609,Secrets Of The Heart,Khalil Gibran,Mysticism,NaN,1993.0,74.0,Available
6807,9788445074879,Fahrenheit 451,Ray Bradbury,Book burning,NaN,2004.0,186.0,Available
6808,9789027712059,The Berlin Phenomenology,Georg Wilhelm Friedrich Hegel,History,Since the three volume edition ofHegel's Philo...,1981.0,210.0,Available


6810

#### Adición de nuevos registros

In [94]:
def add():
    import sqlite3
    try:
        # Conectar a la base de datos
        conn = sqlite3.connect('Biblioteca.db') 
        cursor = conn.cursor()

        # Input para preguntar al usuario
        record_type = input("¿Qué tipo de registro deseas añadir? (book/author/user/loan): ")
        
        if record_type == 'book':
            # Solicitar datos del libro
            isbn = input("ISBN: ")
            title = input("Título: ")
            authors = input("Autores: ")
            categories = input("Categorías: ")
            description = input("Descripción: ")
            published_year = input("Año: ")
            num_pages = input("Número de páginas: ")
            status = input("Estado (Available/Issued): ")

            # Ejecutar la consulta SQL para insertar el libro
            cursor.execute('''
                INSERT INTO books (isbn, title, authors, categories, description, published_year, num_pages, status)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?)
            ''', (isbn, title, authors, categories, description, published_year, num_pages, status))

            print("Libro registrado exitosamente.")
        
        elif record_type == 'author':
            # Solicitar datos del autor
            author_id = input("ID del autor: ")
            names = input("Nombre del autor: ")
            title = input("Título del autor: ")
            published_year = input("Año de publicación: ")

            # Ejecutar la consulta SQL para insertar el autor
            cursor.execute('''
                INSERT INTO authors (authors_id, names, title, published_year)
                VALUES (?, ?, ?, ?)
            ''', (author_id, names, title, published_year))

            print("Autor registrado exitosamente.")
        
        elif record_type == 'user':
            # Solicitar datos del usuario
            user_id = input("ID del usuario: ")
            username = input("Nombre de usuario: ")
            email = input("Correo electrónico: ")
            phone = input("Teléfono: ")
            join_date = input("Fecha de inscripción (YYYY-MM-DD): ")
            status = input("Estado (Active/Inactive): ")

            # Ejecutar la consulta SQL para insertar el usuario
            cursor.execute('''
                INSERT INTO users (user_id, username, email, phone, join_date, status)
                VALUES (?, ?, ?, ?, ?, ?)
            ''', (user_id, username, email, phone, join_date, status))

            print("Usuario registrado exitosamente.")
        
        elif record_type == 'loan':
            # Solicitar datos del préstamo
            isbn = input("ISBN del libro: ")
            user_id = input("ID del usuario: ")
            loan_date = input("Fecha del préstamo (YYYY-MM-DD): ")
            return_date = input("Fecha de devolución (YYYY-MM-DD, déjalo en blanco si no se ha devuelto): ")

            # Ejecutar la consulta SQL para insertar el préstamo
            cursor.execute('''
                INSERT INTO loans (isbn, user_id, loan_date, return_date)
                VALUES (?, ?, ?, ?)
            ''', (isbn, user_id, loan_date, return_date))

            print("Préstamo registrado exitosamente.")
        
        else:
            print("Tipo de registro no válido. Debe ser 'book', 'author', 'user', o 'loan'.")
            return
        # Confirmar los cambios 
        conn.commit()


    except sqlite3.DatabaseError as error:
        print(f"Error de base de datos de SQLite: {error}")

    except Exception as e:
        print(f"Ups, algo no salió bien. Intente todo otra vez. Error: {e}")
    finally:
        # Asegurarse de cerrar la conexión a la base de datos
        if conn:
            conn.close()

add()

Préstamo registrado exitosamente.


#### Eliminación de registros  


In [64]:
#Funcionalidad de eliminación de registros

def delete(): ##Funciona
    try:
        # Conectar a la base de datos
        conn = sqlite3.connect('Biblioteca.db')
        cursor = conn.cursor()

       # Input para preguntar al usuario
        record_type = input("¿Qué tipo de registro deseas eliminar? (book/author/user/loan): ")

        # Definir consulta y solicitar identificador según el tipo de registro, junto con la instrucción de eliminación
        if record_type == 'book':
            identifier = input("ISBN del libro a eliminar: ")
            query = '''DELETE FROM books WHERE isbn = ?'''
        elif record_type == 'author':
            identifier = input("ID del autor a eliminar: ")
            query = '''DELETE FROM authors WHERE authors_id = ?'''
        elif record_type == 'user':
            identifier = input("ID del usuario a eliminar: ")
            query = '''DELETE FROM users WHERE user_id = ?'''
        elif record_type == 'loan':
            identifier = input("ID del préstamo a eliminar: ")
            query = '''DELETE FROM loans WHERE loan_id = ?'''
        else:
            print("Tipo de registro no válido. Debe ser 'book', 'author', 'user', o 'loan'.")
            conn.close()
            return

        # Ejecutar la consulta SQL para eliminar el registro
        cursor.execute(query, (identifier,))

        # Confirmar los cambios
        conn.commit()

        # Verificar si se eliminó alguna fila
        if cursor.rowcount > 0:
            print(f"{record_type.capitalize()} eliminado correctamente.")
        else:
            print("El ID proporcionado no se encuentra en la base de datos.")

    except sqlite3.DatabaseError as error:
        print(f"Error de base de datos de SQLite: {error}")
    except Exception as e:
        print(f"Ups, algo no salió bien. Intente todo otra vez. Error: {e}")
    finally:
        # Asegurarse de cerrar la conexión a la base de datos
        if conn:
            conn.close()

delete()

Error de base de datos de SQLite: no such column: isbn


#### Consulta básica de registros 
 **Objetivo:** Implementar funcionalidades para agregar y obtener registros en la base de datos. 

In [95]:
def query():
    try:
         # Conectar a la base de datos
        conn = sqlite3.connect('Biblioteca.db')
        cursor = conn.cursor()

# Definir consulta y solicitar identificador según el tipo de registro
        record_type = input("¿Qué desea consultar? (book, author, user, loan)")
        if record_type == 'book':
            query = '''SELECT * FROM books'''
        elif record_type == 'author':
            query = '''SELECT * FROM authors'''
        elif record_type == 'user':
            query = '''SELECT * FROM users'''
        elif record_type == 'loan':
            query = '''SELECT * FROM loans'''
        else:
            print("Tipo de registro no válido. Debe ser 'book', 'author', 'user', o 'loan'.")
            return

        # Ejecutar la consulta SQL para eliminar el registro
        cursor.execute(query)

        # Obtener todos los resultados
        registro = cursor.fetchall()

        # Imprimir los resultados
        print('\nRegistros:')
        for i in registro:
            print(i)

    except sqlite3.DatabaseError as error:
        print(f"Error de base de datos de SQLite: {error}")
    except Exception as e:
        print(f"Ups, algo no salió bien. Intente todo otra vez. Error: {e}")
    finally:
        # Asegurarse de cerrar la conexión a la base de datos
        if conn:
            conn.close()

query()


Registros:
(1, 9780002005883, None, '1', '1200-12-12', '1299-12-12')


### Consulta avanzada de registros
 **Objetivo:** Implementar funcionalidades para agregar y obtener registros en la base de datos. (Ser usó listas por compresión)


In [82]:
def advanced_query():
    try:
        # Conectar a la base de datos
        conn = sqlite3.connect('Biblioteca.db')
        cursor = conn.cursor()

        # Definir consulta y solicitar identificador según el tipo de registro
        record_type = input("¿Qué desea consultar? (book, author, user, loan): ")
        
        if record_type == 'book':
            # Solicitar el nombre del libro para filtrar
            book_name = input("Ingrese el nombre del libro o una parte del mismo: ")
            
            # Construir la consulta con WHERE para filtrar por nombre de libro
            query = '''SELECT * FROM books WHERE title LIKE ?'''
            params = (f'%{book_name}%',)
        
        elif record_type == 'author':
            # Solicitar el nombre del autor para filtrar
            author_name = input("Ingrese el nombre del autor o una parte del mismo: ")
            
            # Construir la consulta con WHERE para filtrar por nombre de autor
            query = '''SELECT * FROM authors WHERE names LIKE ?'''
            params = (f'%{author_name}%',)
        
        elif record_type == 'user':
            # Solicitar el nombre del usuario para filtrar
            username = input("Ingrese el nombre del usuario o una parte del mismo: ")
            
            # Construir la consulta con WHERE para filtrar por nombre de usuario
            query = '''SELECT * FROM users WHERE username LIKE ?'''
            params = (f'%{username}%',)
        
        elif record_type == 'loan':
            query = '''SELECT * FROM loans'''
            params = ()
        
        else:
            print("Tipo de registro no válido. Debe ser 'book', 'author', 'user', o 'loan'.")
            return

        # Ejecutar la consulta SQL
        cursor.execute(query, params)

        # Obtener todos los resultados
        registros = cursor.fetchall()

        # Imprimir los resultados
        if registros:
            print('\nRegistros:')
            for registro in registros:
                print(registro)
        else:
            print("No se encontraron registros que coincidan con los filtros proporcionados.")


    except sqlite3.DatabaseError as error:
        print(f"Error de base de datos de SQLite: {error}")
    except Exception as e:
        print(f"Ups, algo no salió bien. Intente todo otra vez. Error: {e}")
    finally:
        # Asegurarse de cerrar la conexión a la base de datos
        if conn:
            conn.close()

# Ejecutar la función
advanced_query()



Títulos de libros encontrados:
['Reading the Bible Again for the First Time', 'The Poisonwood Bible', 'Zondervan Handbook to the Bible', 'Zondervan NIV Study Bible', 'The Everyday Life Bible', 'The Poisonwood Bible', "Reader's digest complete guide to the Bible", 'The Origin of the Bible', 'The Bible Cure for Diabetes', 'The Boomer Bible']


#### Consulta avanzada con uso de listas por compresión

In [ ]:
def advanced_query2():
    try:
        # Conectar a la base de datos
        conn = sqlite3.connect('Biblioteca.db')
        cursor = conn.cursor()

        # Definir consulta y solicitar identificador según el tipo de registro
        record_type = input("¿Qué desea consultar? (book, author, user, loan): ")
        
        if record_type == 'book':
            # Solicitar el nombre del libro para filtrar
            book_name = input("Ingrese el nombre del libro o una parte del mismo: ")
            
            # Construir la consulta con WHERE para filtrar por nombre de libro
            query = '''SELECT * FROM books WHERE title LIKE ?'''
            params = (f'%{book_name}%',)
        
        elif record_type == 'author':
            # Solicitar el nombre del autor para filtrar
            author_name = input("Ingrese el nombre del autor o una parte del mismo: ")
            
            # Construir la consulta con WHERE para filtrar por nombre de autor
            query = '''SELECT * FROM authors WHERE names LIKE ?'''
            params = (f'%{author_name}%',)
        
        elif record_type == 'user':
            # Solicitar el nombre del usuario para filtrar
            username = input("Ingrese el nombre del usuario o una parte del mismo: ")
            
            # Construir la consulta con WHERE para filtrar por nombre de usuario
            query = '''SELECT * FROM users WHERE username LIKE ?'''
            params = (f'%{username}%',)
        
        else:
            print("Tipo de registro no válido. Debe ser 'book', 'author', 'user', o 'loan'.")
            return

        # Ejecutar la consulta SQL
        cursor.execute(query, params)

        # Obtener todos los resultados
        registros = cursor.fetchall()

## Uso de listas de compresión para una mejor respuesta
        if record_type == 'book':
            # Obtener lista de títulos de libros
            titulos = [registro[2] for registro in registros]  
            if titulos:
                print("\nTítulos de libros encontrados:")
                print(titulos)
            else:
                print("No se encontraron libros que coincidan con los filtros proporcionados.")
        
        elif record_type == 'author':
            # Obtener lista de nombres de autores
            nombres_autores = [registro[1] for registro in registros] 
            if nombres_autores:
                print("\nNombres de autores encontrados:")
                print(nombres_autores)
            else:
                print("No se encontraron autores que coincidan con los filtros proporcionados.")
        
        elif record_type == 'user':
            # Filtrar usuarios activos
            usuarios_activos = [registro for registro in registros if registro[6] == 'Active']  
            if usuarios_activos:
                print("\nUsuarios activos encontrados:")
                for usuario in usuarios_activos:
                    print(usuario)
            else:
                print("No se encontraron usuarios activos que coincidan con los filtros proporcionados.")

advanced_query2():

#### Consulta avanzada de registros para préstamos




In [98]:
from collections import Counter

def loan_query():
    try:
        # Conectar a la base de datos
        conn = sqlite3.connect('Biblioteca.db')
        cursor = conn.cursor()

        # Ingresar id del usuario
        user_id = input("Ingrese id del usuario:")

        # Construir la consulta con WHERE para filtrar por id de usuario
        query = '''SELECT * FROM loans WHERE user_id = ?'''
        params = (user_id,)

        # Ejecutar la consulta SQL
        cursor.execute(query, params)
        
        # Obtener todos los resultados
        registros = cursor.fetchall()

        # Verificar si se encontraron registros
        if registros:
            # Usar Counter para contar la cantidad de préstamos por libro
            conteo_prestamos = Counter([registro[1] for registro in registros])  # Asumiendo que el segundo campo es book_id
            print(conteo_prestamos)
        else:
            print("No se encontraron préstamos para este usuario.")

    except Exception as e:
        print(f"Ups, algo no salió bien. Intente otra vez. Error: {e}")

    finally:
        # Cerrar la conexión a la base de datos
        if conn:
            conn.close()

loan_query()


Counter({9780002005883: 1})


In [ ]:
#Agrupar libros por autor utilizando `defaultdict`.
- Usa `defaultdict` para agrupar elementos.

In [ ]:
  - Representar usuarios utilizando `namedtuple`.

- Usa `namedtuple` para crear estructuras de datos más legibles y manejables.- 